In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


In [ ]:
val_dir = '/kaggle/input/landscape-recognition-image-dataset-12k-images/Landscape Classification/Landscape Classification/Validation Data'
test_dir = '/kaggle/input/landscape-recognition-image-dataset-12k-images/Landscape Classification/Landscape Classification/Testing Data'
train_dir = '/kaggle/input/landscape-recognition-image-dataset-12k-images/Landscape Classification/Landscape Classification/Training Data'

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1./255,
)

train_datagen = datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=42
)
val_datagen = datagen.flow_from_directory(
    val_dir,
    target_size=(224,224),
    batch_size=42
)
test_datagen = datagen.flow_from_directory(
    test_dir,
    target_size=(224,244),
    batch_size=42
)


In [ ]:
labels = ['Coast','Desert','Forest','Glacier','Mountains']
num_classes = len(labels)

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
model = InceptionV3(weights='imagenet',include_top=False)
# from keras.layers.serialization import activation
result = model.output 
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
result = GlobalAveragePooling2D()(result)
result = Dense(512,activation='relu')(result)
predictions = Dense(num_classes,activation='sigmoid')(result)

In [ ]:
import tensorflow as tf 
from tensorflow.keras.models import Model

cep_model = Model(inputs=model.input,outputs=predictions)

In [ ]:
cep_model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=['accuracy']
)

In [ ]:
cep_model.summary()

In [ ]:
import tensorflow as tf 

callback =[ 
    tf.keras.callbacks.EarlyStopping(patience=3),
    
          ]
with tf.device("/device:GPU:0"):
    
    history = cep_model.fit_generator(
        train_datagen,
        epochs=50,
        shuffle=True,
        verbose=1,
        validation_data=val_datagen,callbacks = callback)

In [ ]:
print('acc',history.history['accuracy'][-1])
# visualize 
import matplotlib.pyplot as plt
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['loss'])
plt.xlabel('epoch')
plt.legend(['acc','loss'], loc='upper left')
plt.show()

evaluate model acc by test_datagen

In [ ]:
result = cep_model.evaluate(test_datagen)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_datagen,result)